Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-130780
aml-quickstarts-130780
southcentralus
94e14ad4-bf97-47e8-aae0-f9b85a7befa8


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'bank-marketing-experiment'
project_folder = './Users'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
bank-marketing-experiment,quick-starts-ws-130780,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "bank-marketing"
description_text = "This is the Bank Marketing dataset."

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [6]:
dataset.take(5).to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `time_column_name` has to be `cnt` for example.

In [11]:
automl_settings = {
    "experiment_timeout_hours": 1,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset, 
                             label_column_name="y",  
                             path = project_folder,
                             enable_early_stopping= True,
                             model_explainability = True,
                             #featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [12]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [13]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [14]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [15]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [3019b5fd][0c5f2337-57e0-4c40-9984-5631d6621b6b], (This step will run and generate new outputs)
Submitted PipelineRun 355cb335-82c9-4acc-a43d-66c60a4f3f32
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/bank-marketing-experiment/runs/355cb335-82c9-4acc-a43d-66c60a4f3f32?wsid=/subscriptions/94e14ad4-bf97-47e8-aae0-f9b85a7befa8/resourcegroups/aml-quickstarts-130780/workspaces/quick-starts-ws-130780


In [16]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [ ]:
pipeline_run.wait_for_completion()

PipelineRunId: 355cb335-82c9-4acc-a43d-66c60a4f3f32
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/bank-marketing-experiment/runs/355cb335-82c9-4acc-a43d-66c60a4f3f32?wsid=/subscriptions/94e14ad4-bf97-47e8-aae0-f9b85a7befa8/resourcegroups/aml-quickstarts-130780/workspaces/quick-starts-ws-130780
PipelineRun Status: Running


This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [24]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/ceb19e9e-ac33-455a-871d-a82accc30042/metrics_data, 1 files out of an estimated total of 1


In [25]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,ceb19e9e-ac33-455a-871d-a82accc30042_2,ceb19e9e-ac33-455a-871d-a82accc30042_28,ceb19e9e-ac33-455a-871d-a82accc30042_8,ceb19e9e-ac33-455a-871d-a82accc30042_9,ceb19e9e-ac33-455a-871d-a82accc30042_21,ceb19e9e-ac33-455a-871d-a82accc30042_23,ceb19e9e-ac33-455a-871d-a82accc30042_36,ceb19e9e-ac33-455a-871d-a82accc30042_22,ceb19e9e-ac33-455a-871d-a82accc30042_20,ceb19e9e-ac33-455a-871d-a82accc30042_27,...,ceb19e9e-ac33-455a-871d-a82accc30042_13,ceb19e9e-ac33-455a-871d-a82accc30042_16,ceb19e9e-ac33-455a-871d-a82accc30042_32,ceb19e9e-ac33-455a-871d-a82accc30042_7,ceb19e9e-ac33-455a-871d-a82accc30042_12,ceb19e9e-ac33-455a-871d-a82accc30042_11,ceb19e9e-ac33-455a-871d-a82accc30042_33,ceb19e9e-ac33-455a-871d-a82accc30042_29,ceb19e9e-ac33-455a-871d-a82accc30042_4,ceb19e9e-ac33-455a-871d-a82accc30042_14
spearman_correlation,[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],...,[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0]
median_absolute_error,[1159.2],[394.1212121212121],[291.91960248214116],[1.089808483811794e-06],[268.975873015873],[172.91666666666666],[1.089808483811794e-06],[408.0833333333333],[476.05795155734234],[688.1718253968254],...,[260.8184210526316],[330.8071428571429],[0.0007882542991865194],[462.2830848861283],[2.5613256100781654],[0.001339420246586087],[1.1701537581160664],[36.38990612096286],[2298.8109825547426],[82.36666666666665]
normalized_root_mean_squared_error,[0.13336401288541186],[0.04534298344698713],[0.03358485992661541],[1.2538063550526851e-10],[0.030945222390229288],[0.019893772050928054],[1.2538063550526851e-10],[0.04694930204019021],[0.054769667689523974],[0.079173012585921],...,[0.030006721243975097],[0.03805880612714484],[9.0687333086346e-08],[0.05318489241671979],[0.00029467620916683913],[1.5409804953820608e-07],[0.0001346242243575778],[0.004186597574892184],[0.26447434221752675],[0.009476146648258934]
mean_absolute_error,[1159.2],[394.1212121212121],[291.91960248214116],[1.089808483811794e-06],[268.975873015873],[172.91666666666666],[1.089808483811794e-06],[408.0833333333333],[476.05795155734234],[688.1718253968254],...,[260.8184210526316],[330.8071428571429],[0.0007882542991865194],[462.2830848861283],[2.5613256100781654],[0.001339420246586087],[1.1701537581160664],[36.38990612096286],[2298.8109825547426],[82.36666666666665]
explained_variance,[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],...,[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0]
root_mean_squared_log_error,[0.6985293680914578],[0.1833085240761287],[0.1497535928612118],[5.849651429912228e-10],[0.1908900889615129],[0.07961041460925386],[5.849651429912228e-10],[0.15916440373222987],[0.2213243248618836],[0.33802039983478044],...,[0.09692594665988885],[0.15887267311422165],[4.103812390354733e-07],[0.21896016060981777],[0.0009864546254579666],[7.149066021838734e-07],[0.0004971920372703664],[0.021106884024354143],[0.7546133762185778],[0.046621384821859554]
normalized_root_mean_squared_log_error,[0.11765088085262347],[0.03087401949079857],[0.02522247870423097],[9.852365195329546e-11],[0.032150956192033994],[0.013408506258529945],[9.852365195329546e-11],[0.026807508967937443],[0.03727688908092342],[0.0569316046014991],...,[0.0163249013182754],[0.026758373790880936],[6.911908999593124e-08],[0.03687870108849041],[0.00016614513420297488],[1.2040924163875155e-07],[8.374033191702585e-05],[0.003554959334500453],[0.12709691599346445],[0.00785227828838079]
normalized_median_absolute_error,[0.13336401288541186],[0.04534298344698713],[0.03358485992661541],[1.2538063550526851e-10],[0.030945222390229288],[0.019893772050928054],[1.2538063550526851e-10],[0.04694930204019021],[0.054769667689523974],[0.079173012585921],...,[0.030006721243975097],[0.03805880612714484],[9.0687333086346e-08],[0.05318489241671979],[0.00029467620916683913],[1.5409804953820608e-07],[0.0001346242243575778],[0.004186597574892184],[0.26447434221752675],[0.009476146648258934]
normalized_mean_absolute_error,[0.13336401288541186]

### Retrieve the Best Model

In [26]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/ceb19e9e-ac33-455a-871d-a82accc30042/model_data, 1 files out of an estimated total of 1


In [27]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

ForecastingPipelineWrapper(pipeline=Pipeline(memory=None,
                                             steps=[('timeseriestransformer',
                                                     TimeSeriesTransformer(featurization_config=None,
                                                                           pipeline_type=<TimeSeriesPipelineType.FULL: 1>)),
                                                    ('StandardScalerWrapper',
                                                     <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7fc50a5b6f60>),
                                                    ('LassoLars',
                                                     LassoLars(alpha=0.001,
                                                               copy_X=True,
                                                               eps=2.220446049250313e-16,
                                                               fit_intercept=True,
                   

In [28]:
best_model.steps

[('timeseriestransformer',
  TimeSeriesTransformer(featurization_config=None,
                        pipeline_type=<TimeSeriesPipelineType.FULL: 1>)),
 ('StandardScalerWrapper',
  <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper at 0x7fc50a5b6f60>),
 ('LassoLars',
  LassoLars(alpha=0.001, copy_X=True, eps=2.220446049250313e-16,
            fit_intercept=True, fit_path=True, max_iter=500, normalize=False,
            positive=False, precompute='auto', verbose=False))]

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [29]:
# Ensure that the workspace is once again available
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-127942
aml-quickstarts-127942
southcentralus
82648f26-b738-43a4-9ebb-f954c9f1ff3a


In [30]:
# NOTE: update these to match your existing experiment name and a previous experiment
experiment_name = 'bank-marketing'
project_folder = './Exercise_starter_files'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-bike-experiment-1,quick-starts-ws-127942,Link to Azure Machine Learning studio,Link to Documentation


In [31]:
from azureml.pipeline.core import PipelineRun

run_id = "26cabd57-d59d-47a6-80e3-0ae5934e834d" #pick run id from experiment
pipeline_run = PipelineRun(experiment, run_id)

ServiceException: ServiceException:
	Code: 404
	Message: (UserError) Run 78e729c3-4746-417f-ad9a-abe970f4966f was not found
	Details:

	Headers: {
	    "Date": "Thu, 26 Nov 2020 07:22:24 GMT",
	    "Content-Type": "application/json; charset=utf-8",
	    "Transfer-Encoding": "chunked",
	    "Connection": "keep-alive",
	    "Vary": "Accept-Encoding",
	    "Request-Context": "appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d",
	    "x-ms-response-type": "error",
	    "x-ms-client-request-id": "03585bd6-60a9-45ed-9eed-c6feeed6978d",
	    "x-ms-client-session-id": "",
	    "X-Content-Type-Options": "nosniff",
	    "x-request-time": "0.048",
	    "Strict-Transport-Security": "max-age=15724800; includeSubDomains; preload",
	    "Content-Encoding": "gzip"
	}
	InnerException: {
    "additional_properties": {},
    "error": {
        "additional_properties": {
            "debugInfo": null
        },
        "code": "UserError",
        "severity": null,
        "message": "Run 78e729c3-4746-417

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name="bank-marketing", description="Training bank marketing pipeline", version="1.0")

published_pipeline


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-bankMarketing-rest-endpoint"}
                        )

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-bankMarketing-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()